General GLM for classification:
* https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-logistic.html


An example of Spike and slab:
* https://www.kaggle.com/code/melondonkey/bayesian-spike-and-slab-in-pymc3/notebook


An example with adding priors:
* https://discourse.pymc.io/t/linear-regression-with-positivity-constraint/2598/4?u=junpenglao
* https://discourse.pymc.io/t/glm-logistic-regression-with-custom-prior-in-pymc3-v-3-6/2644/3

Generating out of sample predictions:
* https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-out-of-sample-predictions.html
* https://bpostance.github.io/posts/pymc3-predictions/





In [ ]:
import warnings

from collections import OrderedDict
from time import time

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn
import theano as thno
import theano.tensor as T

from scipy import integrate
from scipy.optimize import fmin_powell

print(f"Running on PyMC3 v{pm.__version__}")

warnings.filterwarnings("ignore")

In [ ]:
def run_models(df, upper_order=5):
    """
    Convenience function:
    Fit a range of pymc3 models of increasing polynomial complexity.
    Suggest limit to max order 5 since calculation time is exponential.
    """

    models, traces = OrderedDict(), OrderedDict()

    for k in range(1, upper_order + 1):

        nm = f"k{k}"
        fml = create_poly_modelspec(k)

        with pm.Model() as models[nm]:

            print(f"\nRunning: {nm}")
            pm.glm.GLM.from_formula(fml, df, family=pm.glm.families.Binomial())

            traces[nm] = pm.sample(1000, tune=1000, init="adapt_diag", return_inferencedata=True)

    return models, traces


def plot_traces(traces, model, retain=0):
    """
    Convenience function:
    Plot traces with overlaid means and values
    """
    with model:
        ax = az.plot_trace(
            traces[-retain:],
            lines=tuple([(k, {}, v["mean"]) for k, v in az.summary(traces[-retain:]).iterrows()]),
        )

        for i, mn in enumerate(az.summary(traces[-retain:])["mean"]):
            ax[i, 0].annotate(
                f"{mn:.2f}",
                xy=(mn, 0),
                xycoords="data",
                xytext=(5, 10),
                textcoords="offset points",
                rotation=90,
                va="bottom",
                fontsize="large",
                color="#AA0022",
            )


def create_poly_modelspec(k=1):
    """
    Convenience function:
    Create a polynomial modelspec string for patsy
    """
    return (
        "income ~ educ + hours + age " + " ".join([f"+ np.power(age,{j})" for j in range(2, k + 1)])
    ).strip()

In [ ]:
raw_data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None,
    names=[
        "age",
        "workclass",
        "fnlwgt",
        "education-categorical",
        "educ",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "captial-gain",
        "capital-loss",
        "hours",
        "native-country",
        "income",
    ],
)

In [ ]:
raw_data.head(10)

In [ ]:
data = raw_data[~pd.isnull(raw_data["income"])]

In [ ]:
data[data["native-country"] == " United-States"].sample(5)

In [ ]:
income = 1 * (data["income"] == " >50K")

In [ ]:
data = data[["age", "educ", "hours"]]

# Scale age by 10, it helps with model convergence.
data["age"] = data["age"] / 10.0
data["age2"] = np.square(data["age"])
data["income"] = income

In [ ]:
income.value_counts()

In [ ]:
g = seaborn.pairplot(data)

In [ ]:
# Compute the correlation matrix
corr = data.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = seaborn.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
seaborn.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.3,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    ax=ax,
);

In [ ]:
with pm.Model() as logistic_model:
    pm.glm.GLM.from_formula(
        "income ~ age + age2 + educ + hours", data, family=pm.glm.families.Binomial()
    )
    trace = pm.sample(1000, tune=1000, init="adapt_diag")

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
plot_traces(trace, logistic_model);

In [ ]:
plt.figure(figsize=(9, 7))
seaborn.jointplot(trace["age"], trace["educ"], kind="hex", color="#4CB391")
plt.xlabel("beta_age")
plt.ylabel("beta_educ");

In [ ]:
def lm_full(trace, age, educ, hours):
    shape = np.broadcast(age, educ, hours).shape
    x_norm = np.asarray([np.broadcast_to(x, shape) for x in [age / 10.0, educ, hours]])
    return 1 / (
        1
        + np.exp(
            -(
                trace["Intercept"]
                + trace["age"] * x_norm[0]
                + trace["age2"] * (x_norm[0] ** 2)
                + trace["educ"] * x_norm[1]
                + trace["hours"] * x_norm[2]
            )
        )
    )


# Linear model with hours == 50 and educ == 12
lm = lambda x, samples: lm_full(samples, x, 12.0, 50.0)

# Linear model with hours == 50 and educ == 16
lm2 = lambda x, samples: lm_full(samples, x, 16.0, 50.0)

# Linear model with hours == 50 and educ == 19
lm3 = lambda x, samples: lm_full(samples, x, 19.0, 50.0)

In [ ]:
# Plot the posterior predictive distributions of P(income > $50K) vs. age
pm.plot_posterior_predictive_glm(
    trace, eval=np.linspace(25, 75, 1000), lm=lm, samples=100, color="blue", alpha=0.15
)
pm.plot_posterior_predictive_glm(
    trace,
    eval=np.linspace(25, 75, 1000),
    lm=lm2,
    samples=100,
    color="green",
    alpha=0.15,
)
pm.plot_posterior_predictive_glm(
    trace, eval=np.linspace(25, 75, 1000), lm=lm3, samples=100, color="red", alpha=0.15
)

import matplotlib.lines as mlines

blue_line = mlines.Line2D(["lm"], [], color="b", label="High School Education")
green_line = mlines.Line2D(["lm2"], [], color="g", label="Bachelors")
red_line = mlines.Line2D(["lm3"], [], color="r", label="Grad School")
plt.legend(handles=[blue_line, green_line, red_line], loc="lower right")
plt.ylabel("P(Income > $50K)")
plt.xlabel("Age")
plt.show()

In [ ]:
b = trace["educ"]
plt.hist(np.exp(b), bins=20, density=True)
plt.xlabel("Odds Ratio")
plt.show()

In [ ]:
lb, ub = np.percentile(b, 2.5), np.percentile(b, 97.5)

print("P({:.3f} < O.R. < {:.3f}) = 0.95".format(np.exp(lb), np.exp(ub)))

In [ ]:
models_lin, traces_lin = run_models(data, 3)

In [ ]:
model_trace_dict = dict()
for nm in ["k1", "k2", "k3"]:
    model_trace_dict.update({nm: traces_lin[nm]})

dfwaic = az.compare(model_trace_dict, ic="WAIC", scale="deviance")
az.plot_compare(dfwaic);

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w